In [6]:
import googlemaps

In [7]:
import gmaps
from pyproj import Proj
from shapely.geometry import Polygon

In [8]:
#get api_key
with open("api_key.txt") as f:
    api_key = f.read().splitlines()[0]
    f.close

In [9]:
#configure gmaps library with api_key
gmaps.configure(api_key=api_key)
mymaps = googlemaps.Client(key=api_key)

In [10]:
#get center from address
print("enter address")
address = input()

enter address
world trade center, new york


### Run everything above

In [11]:
center_dict = mymaps.geocode(address)[0]['geometry']['location']
center = center_dict.values()

In [12]:
fig = gmaps.figure(zoom_level=20, center = center, layout={'width': '600px', 'height': '400px'})
#add drawing layer
drawing = gmaps.drawing_layer()
fig.add_layer(drawing)


In [48]:
print("Select corners of the polygon")
fig

Select corners of the polygon


Figure(layout=FigureLayout(height='400px', width='600px'))

### Run everything above

In [49]:
point_list = []
for point in drawing.features:
    point_list.append(point.location)
    
point_list

#marker is in lat,long format

[(40.71134722822828, -74.0132047489455),
 (40.7111296831432, -74.01268708260562),
 (40.710733220646155, -74.01296603234317),
 (40.71095076702654, -74.01349979193714),
 (40.712052383366945, -74.01363403357288),
 (40.711800277648486, -74.0130600408437),
 (40.712245528231264, -74.0127354935529),
 (40.71249495695026, -74.01330860391352)]

In [50]:
# switch points to (long,lat) form
lnglat_point_list  = []

for point in point_list:
    lnglat_point_list.append(tuple(reversed(point)))
    
lnglat_point_list

[(-74.0132047489455, 40.71134722822828),
 (-74.01268708260562, 40.7111296831432),
 (-74.01296603234317, 40.710733220646155),
 (-74.01349979193714, 40.71095076702654),
 (-74.01363403357288, 40.712052383366945),
 (-74.0130600408437, 40.711800277648486),
 (-74.0127354935529, 40.712245528231264),
 (-74.01330860391352, 40.71249495695026)]

##### Using a UTM projection
###### Universal Transverse Mercator 
Area is distorted, but distortions are minimal as long as the area is within a single UTM zone


In [51]:
#Need to find UTM zone
def get_utm_zone(geotuple):
    '''
    get utm zone based on geographic coordinates
    Input geo coords - tuple (longitude, latitute)
    output utm zone
    '''
    long = round(geotuple[0])
    zone = ((180+long)//6)+1
    utm_zone = str(zone)
    
    lat = geotuple[1]
    
    if lat<0:
        utm_zone += " +south"
    
    return utm_zone

In [52]:
utm_zone = get_utm_zone(lnglat_point_list[0])
utm_zone

'18'

In [53]:
#Project geocords to 2d plane using UTM projection
proj_string = "+proj=utm +zone="+utm_zone
proj_string

'+proj=utm +zone=18'

In [54]:
p = Proj(proj_string)

In [55]:
#split lnglat_point_list into two separate lists of lat and lng
lng_list,lat_list = zip(*lnglat_point_list)
    

In [56]:
x,y=p(list(lng_list),list(lat_list))

In [57]:
xy_list = list(zip(x,y))
xy_list

[(583352.6174560806, 4507182.869033737),
 (583396.6157960166, 4507159.21152254),
 (583373.5475934539, 4507114.936828562),
 (583328.1896786235, 4507138.579353779),
 (583315.4770638918, 4507260.7384771425),
 (583364.2757491431, 4507233.297759017),
 (583391.1342796274, 4507283.03151907),
 (583342.4136804501, 4507310.175732358)]

### calc area using shapely

In [58]:
#cast coordinates as polygon
from shapely.geometry import Polygon
polygon = Polygon(xy_list)

In [59]:
#area of polygon
m2_area = polygon.area #Area in meter-square
acreage = m2_area*0.000247105
sqft =  m2_area*10.7639


print("Area is {} square-m or {} acres or {} square-ft".format(m2_area,acreage,sqft))

Area is 3128.598824017922 square-m or 0.7730924124089487 acres or 33675.92488184651 square-ft
